## All my life, I've been searching for something

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import re
import numpy as np

In [2]:
pattern = 'Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)'

pattern = re.compile(pattern)

beacons = [[sx, sy, bx, by, abs(sx-bx) + abs(sy-by)]
           for sx, sy, bx, by in [[int(coord) for coord in re.search(pattern, line).groups()] for line in Path('beacons.txt').read_text().split('\n')]]

beacon_locations = [[bx, by] for sx, sy, bx, by, dist in beacons]
sensor_locations = [[sx, sy] for sx, sy, bx, by, dist in beacons]

In [3]:
target_row = 2000000

forbidden_xcoords = []

for sx, sy, bx, by, dist in beacons:
    remaining_dist = dist - abs(target_row - sy)
    if remaining_dist < 0:
        continue
    else:
        forbidden_xcoords += [x for x in range(sx-remaining_dist, sx + remaining_dist + 1) if [x, target_row] not in beacon_locations and [x, target_row] not in sensor_locations]
        
print(len(set(forbidden_xcoords)))

5403290


## Part 2: Nothing satisfies but I'm getting close

In [ ]:
# Every sensor, beacon combo gives the following four conditions:
# x + y > dist + sx + sy
# x + y < -dist + sx + sy
# x - y > dist + sx - sy
# x - y < -dist + sx - sy

# Allowed region for beacon is the union of these four conditions

# The 8 neighboring points to the unique solution must be on the edge of vertex of a region
# Unique solution is at minimum dist + 1 from a sensor

# Moving from arbitrary point (x0, y0) diagonally towards a boundary, the length t of the vector is:

# 2t > dist + sx + sy - x0 - y0
# 2t > dist - sx - sy + x0 + y0
# 2t > dist + sx - sy - x0 + y0
# 2t > dist - sx + sy + x0 - y0

# The four points (x1, y1) of possible travel are:
# x1 = x0 + (dist + sx + sy - x0 - y0)/2, y1 = y0 + (dist + sx + sy - x0 - y0)/2
# x1 = x0 + (dist - sx - sy + x0 + y0)/2, y1 = y0 + (dist - sx - sy + x0 + y0)/2
# x1 = x0 + (dist + sx - sy - x0 + y0)/2, y1 = y0 + (dist + sx - sy - x0 + y0)/2
# x1 = x0 + (dist - sx + sy + x0 - y0)/2, y1 = y0 + (dist - sx + sy + x0 - y0)/2

In [170]:
def travel(x, y, beacons, xymax = 4000000):
    inside = [[sx, sy, dist] for sx, sy, bx, by, dist in beacons if dist - abs(sx-x) - abs(sy-y) >= 0]
    
    if len(inside) == 0:
        return ['Found it!', x, y]
    
    points = []
    
    for sx, sy, dist in inside:

        a = sx + sy - x - y
        b = sx - sy - x + y

        vels = [[1, 1], [-1, -1], [1, -1], [-1, 1]]

        num_steps = [(dist + a)//2 + 1, (dist - a)//2 + 1, (dist + b)//2 + 1, (dist - b)//2 + 1]

        new_coords = [[x + ns*vel[0], y + ns*vel[1]] for ns, vel in zip(num_steps, vels)]

        filtered = [[nx, ny] for nx, ny in new_coords if nx >= 0 and nx <= xymax and ny >= 0 and ny <= xymax]
        
        distances = [(abs(sx - nx) + abs(sy - ny) - dist) for nx, ny in filtered]
        
        if np.all(np.array(distances) == 2):
            shifts = [[1, 0], [-1, 0], [0, 1], [0, -1]]
            filtered = [[nx+shift[0], ny + shift[1]] for nx, ny in filtered for shift in shifts 
                    if abs(sx - nx - shift[0]) + abs(sy - ny - shift[1]) == dist + 1]
            
            distances = [(abs(sx - nx) + abs(sy - ny) - dist) for nx, ny in filtered]
            
            assert(np.all(np.array(distances)) == 1)
        
        points += [point for point in filtered if filtered not in points]
    
    return points

In [174]:
xymax = 4000000

prev_visited = [[0, 0]]

new_points = {0: prev_visited}

found = False

layer = 0

while not found:
    
    new_points[layer+1] = []
    
    for x, y in new_points[layer]:
        
        possibly_new = travel(x, y, beacons)
        
        if possibly_new[0] == 'Found it!':
            found = True
            foundx, foundy = possibly_new[1:]
            break
        
        actually_new = [new for new in possibly_new if new not in prev_visited]
        
        prev_visited += actually_new
    
        new_points[layer+1] += actually_new
        
    layer += 1
    
print(foundx*xymax + foundy)

10291582906626


## Then I'm done, done, on to the next one